In [1]:
import numpy as np
import pandas as pd
import ast
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler
import pickle

In [2]:
def convert(text):
    L = []
    for i in ast.literal_eval(text):
        L.append(i['name']) 
    return L

In [3]:
def fetch_director(text):
    L = []
    for i in ast.literal_eval(text):
        if i['job'] == 'Director':
            L.append(i['name'])
    return L

In [4]:
def stem(text):
    ps = PorterStemmer()
    L = []
    for i in text.split():
        L.append(ps.stem(i))
    return ' '.join(L)

In [5]:
def weighted_vote_average(record):
    return ((record['vote_count'] * record['vote_average']) + (movies['vote_count'].quantile(0.6) * movies['vote_average'].mean()))/(movies['vote_count'].quantile(0.6) + record['vote_count'])

In [6]:
movies = pd.read_csv('tmdb_5000_movies.csv')
credits = pd.read_csv('tmdb_5000_credits.csv')

In [7]:
movies = movies.merge(credits, on = 'title')

In [8]:
movies = movies[['movie_id', 'title', 'overview', 'genres', 'keywords', 'cast', 'crew', 'popularity', 'revenue', 'vote_average', 'vote_count']]
movies['weighted_vote'] = movies.apply(weighted_vote_average, axis=1)

In [9]:
movies.drop(['vote_average', 'vote_count'], axis = 'columns', inplace = True)
movies.dropna(inplace = True)

In [10]:
movies['genres'] = movies['genres'].apply(convert)
movies['keywords'] = movies['keywords'].apply(convert)
movies['cast'] = movies['cast'].apply(convert)
movies['cast'] = movies['cast'].apply(lambda x: x[0: 3])
movies['crew'] = movies['crew'].apply(fetch_director)

In [11]:
movies['genres'] = movies['genres'].apply(lambda x: [i.replace(' ', '') for i in x])
movies['keywords'] = movies['keywords'].apply(lambda x: [i.replace(' ', '') for i in x])
movies['cast'] = movies['cast'].apply(lambda x: [i.replace(' ', '') for i in x])
movies['crew'] = movies['crew'].apply(lambda x: [i.replace(' ', '') for i in x])
movies['overview'] = movies['overview'].apply(lambda x: x.split())

In [12]:
movies['tags'] = movies['overview'] + movies['genres'] + movies['keywords'] + movies['cast'] + movies['crew']

In [13]:
new = movies.drop(columns = ['overview', 'genres', 'keywords', 'cast', 'crew'])
new['tags'] = new['tags'].apply(lambda x: ' '.join(x))
new['tags'] = new['tags'].apply(lambda x: x.lower())

In [14]:
new['tags'] = new['tags'].apply(stem)

In [15]:
cv = CountVectorizer(max_features = 5000, stop_words = 'english')

In [16]:
tag_vector = cv.fit_transform(new['tags']).toarray()

In [17]:
mms = MinMaxScaler(feature_range = (0, 1))

In [18]:
numeric_feature_vector = mms.fit_transform(new[['weighted_vote', 'revenue', 'popularity']])

In [19]:
final_vector = np.concatenate((tag_vector, numeric_feature_vector), axis = 1)

In [20]:
similarity = cosine_similarity(final_vector)

In [21]:
pickle.dump(new, open('movie_list.pkl', 'wb'))
pickle.dump(similarity, open('similarity.pkl', 'wb'))